In [1]:
import pandas as pd
import numpy as np
import math

#### Read Mutant Original Data

In [2]:
eval_dir = "gc_imdb"
df = pd.read_csv("../data/" + eval_dir + "/test.csv", header=None, sep="\t", names=["label", "mutant", "gender", "template", "name", "template_id"])
df

,label,mutant,gender,template,name,template_id
0,1,"I have only see three episodes of Hack, starri...",template,"I have only see three episodes of Hack, starri...",template,1846
1,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",Roen,1846
2,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",Jeet,1846
3,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",Hagen,1846
4,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",Willow,1846
...,...,...,...,...,...,...
1891808,1,"First, I'm a huge Dódi fan. I grew up knowing ...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",Dódi,1128
1891809,1,"First, I'm a huge Iboyka fan. I grew up knowin...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",Iboyka,1128
1891810,1,"First, I'm a huge Zsófia fan. I grew up knowin...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",Zsófia,1128
1891811,1,"First, I'm a huge Ágota fan. I grew up knowing...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",Ágota,1128


#### Read Mutant Prediction

In [3]:
def read_txt(fpath):
    pred = []
    file = open(fpath)
    lines = file.readlines()
    for l in lines :
        pred.append(int(l))
    file.close()
    
    return pred

In [4]:
output_dir = "gc_imdb"

result_dir = "../result/" + output_dir + "/"

path = result_dir + "results_data.txt"

pred = read_txt(path)

print(len(pred))

1891813


In [5]:
df["prediction"] = pred

#### Separate the Original Text with Mutant Text

In [10]:
dft = df[df["gender"] == "template"]
df = df[df["gender"] != "template"]

# dft = df[df.index % 311 == 0]
# df = df[df.index % 311 != 0]

## template
dft = dft.reset_index(drop=True)

## mutant
df = df.reset_index(drop=True)

In [11]:
## template only


dft = dft.drop(columns=["mutant", "gender"])

# sort by template_id and reset index to faster performance when grouping by template_id and processing on it 
dft = dft.sort_values(["template_id"])
dft = dft.reset_index(drop=True)
dft.head()

,label,template,name,template_id,prediction


In [12]:
# mutant only

df = df.sort_values(["template_id", "gender"])
df = df.reset_index(drop=True)
df.head()

,label,mutant,gender,template,name,template_id,prediction
0,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,Sárinka,0,0
1,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,Elenka,0,0
2,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,Leona,0,0
3,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,Katuška,0,0
4,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,Miladka,0,0


## Get Number of Discordant Pairs for Each Template

There is a memory limitation that make us can't directly produce +- 240M pairs. Fortunately, the number of discordant pairs for each template can be calculate theoritically without crossing th data to get +- 240M pairs. This will solve the memory issue.

For each template, we will give an example of the male mutant and female mutant for user study

In [13]:
gb = df.groupby("template_id")
gb.count()

,label,mutant,gender,template,name,prediction
template_id,,,,,,
0,310,310,310,310,310,310
1,310,310,310,310,310,310
2,310,310,310,310,310,310
3,310,310,310,310,310,310
4,310,310,310,310,310,310
...,...,...,...,...,...,...
6078,310,310,310,310,310,310
6079,310,310,310,310,310,310
6080,310,310,310,310,310,310


## Create Rank based on the name on the false positive template

#### Get false positive template 

In [23]:
df_fp = pd.read_csv("../data/dp/labelled-full.csv", encoding="latin1")

In [42]:
df_fp

,random_index,sorted_index,template_id,gender,text,is_make_sense,sentiment,comment
0,568,0,3495,t,"OK, maybe it doesn't deserve an Oscar. Or a Go...",Yes,Positive,NaN
1,483,1,3495,m,"OK, maybe it doesn't deserve an Oscar. Or a Go...",No,Positive,Male do strip-tease
2,152,2,3495,f,"OK, maybe it doesn't deserve an Oscar. Or a Go...",Yes,Positive,NaN
3,540,3,1681,t,"I find it terribly ironic that ""left wing"" Hol...",Yes,Negative,NaN
4,3,4,1681,m,"I find it terribly ironic that ""left wing"" Hol...",Yes,Negative,NaN
...,...,...,...,...,...,...,...,...
694,147,694,1393,m,"Hilary was great as julie, and Pat was once ag...",Yes,Positive,NaN
695,250,695,1393,f,"Hilary was great as julie, and Pat was once ag...",No,Positive,"Mr. Miyagi, Lotte was ""her"" supposed to be his"
696,554,696,228,t,A party-hardy frat boy's sister is brutally mu...,Yes,Negative,NaN
697,698,697,228,m,A party-hardy frat boy's sister is brutally mu...,Yes,Negative,NaN


In [43]:
# not make sense
nms = df_fp[df_fp["is_make_sense"] == "No"]

In [44]:
nms_count = nms.groupby("template_id").count()

nms_count.tail()

,random_index,sorted_index,gender,text,is_make_sense,sentiment,comment
template_id,,,,,,,
5280,2,2,2,2,2,2,2
5665,1,1,1,1,1,1,1
5678,1,1,1,1,1,1,1
5723,1,1,1,1,1,1,1
5829,1,1,1,1,1,1,1


In [45]:
# not make sense 1 for each template only
nmsot = nms_count[nms_count["text"] == 1]
nmsot.tail()

,random_index,sorted_index,gender,text,is_make_sense,sentiment,comment
template_id,,,,,,,
5235,1,1,1,1,1,1,1
5665,1,1,1,1,1,1,1
5678,1,1,1,1,1,1,1
5723,1,1,1,1,1,1,1
5829,1,1,1,1,1,1,1


In [46]:
len(nmsot)

36

In [47]:
# false positive index
fp_index = nmsot.index
fp_index

Int64Index([  51,   90,  228,  629,  664,  727,  789, 1117, 1393, 2074, 2126,
            2378, 2424, 2617, 2751, 2842, 2890, 2957, 3130, 3256, 3329, 3495,
            3524, 3559, 3769, 3922, 3968, 4085, 5020, 5063, 5205, 5235, 5665,
            5678, 5723, 5829],
           dtype='int64', name='template_id')

In [48]:
def is_in_fp_index(id) :
    return id in fp_index

# get valid invalid mutant
dfvi = df_fp[df_fp["template_id"].apply(is_in_fp_index)]
dfvi.head()

,random_index,sorted_index,template_id,gender,text,is_make_sense,sentiment,comment
0,568,0,3495,t,"OK, maybe it doesn't deserve an Oscar. Or a Go...",Yes,Positive,NaN
1,483,1,3495,m,"OK, maybe it doesn't deserve an Oscar. Or a Go...",No,Positive,Male do strip-tease
2,152,2,3495,f,"OK, maybe it doesn't deserve an Oscar. Or a Go...",Yes,Positive,NaN
12,258,12,51,t,(Warning: Some spoilers ahead.) What an incred...,Yes,Positive,NaN
13,608,13,51,m,(Warning: Some spoilers ahead.) What an incred...,Yes,Positive,NaN


In [38]:
# male
mvi = dfvi.groupby("gender").get_group("m")

# female
fvi = dfvi.groupby("gender").get_group("f")
fvi.head()

,random_index,sorted_index,template_id,gender,text,is_make_sense,sentiment,comment
2,152,2,3495,f,"OK, maybe it doesn't deserve an Oscar. Or a Go...",Yes,Positive,NaN
14,82,14,51,f,(Warning: Some spoilers ahead.) What an incred...,No,Positive,her pregnant wife
17,271,17,2957,f,It's fun to watch Eugenia doing her thing. She...,No,Positive,even when rejected by her wife to be
20,340,20,664,f,"Before going any further, I have to admit that...",Yes,Positive,NaN
44,610,44,3922,f,"Snap, crackle, pop! The jarring sound of every...",No,Positive,laid-back young man -> ÐÑÐ»Ð¸ÑÑÐ° is woman


In [72]:
def get_male_is_make_sense(id):
    return mvi[mvi["template_id"] == id]["is_make_sense"].values[0]

def get_female_is_make_sense(id):
    return fvi[fvi["template_id"] == id]["is_make_sense"].values[0]

get_male_is_make_sense(3495)

'No'

#### Filter data based on the template_id

In [49]:
df.head()

,label,mutant,gender,template,name,template_id,prediction
0,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,Sárinka,0,0
1,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,Elenka,0,0
2,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,Leona,0,0
3,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,Katuška,0,0
4,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,Miladka,0,0


In [50]:
def is_in_fp_index(id) :
    return id in fp_index

# selected df
sdf = df[df["template_id"].apply(is_in_fp_index)]

In [51]:
sdf = sdf.reset_index(drop=True)
sdf

,label,mutant,gender,template,name,template_id,prediction
0,0,(Warning: Some spoilers ahead.) What an incred...,female,(Warning: Some spoilers ahead.) What an incred...,Sárinka,51,0
1,0,(Warning: Some spoilers ahead.) What an incred...,female,(Warning: Some spoilers ahead.) What an incred...,Elenka,51,0
2,0,(Warning: Some spoilers ahead.) What an incred...,female,(Warning: Some spoilers ahead.) What an incred...,Leona,51,0
3,0,(Warning: Some spoilers ahead.) What an incred...,female,(Warning: Some spoilers ahead.) What an incred...,Katuška,51,0
4,0,(Warning: Some spoilers ahead.) What an incred...,female,(Warning: Some spoilers ahead.) What an incred...,Miladka,51,0
...,...,...,...,...,...,...,...
11155,1,"Wow what can I say it was a good movie, very d...",male,"Wow what can I say it was a good movie, very d...",Cedrick,5829,1
11156,1,"Wow what can I say it was a good movie, very d...",male,"Wow what can I say it was a good movie, very d...",Rosendo,5829,1
11157,1,"Wow what can I say it was a good movie, very d...",male,"Wow what can I say it was a good movie, very d...",Quinton,5829,1
11158,1,"Wow what can I say it was a good movie, very d...",male,"Wow what can I say it was a good movie, very d...",Devon,5829,0


In [52]:
sgb = sdf.groupby("template_id")
sgb.count()

,label,mutant,gender,template,name,prediction
template_id,,,,,,
51,310,310,310,310,310,310
90,310,310,310,310,310,310
228,310,310,310,310,310,310
629,310,310,310,310,310,310
664,310,310,310,310,310,310
727,310,310,310,310,310,310
789,310,310,310,310,310,310
1117,310,310,310,310,310,310
1393,310,310,310,310,310,310


In [53]:
sgb.count().index.values

array([  51,   90,  228,  629,  664,  727,  789, 1117, 1393, 2074, 2126,
       2378, 2424, 2617, 2751, 2842, 2890, 2957, 3130, 3256, 3329, 3495,
       3524, 3559, 3769, 3922, 3968, 4085, 5020, 5063, 5205, 5235, 5665,
       5678, 5723, 5829])

In [54]:
len(sgb.count().index.values)

36

In [55]:
import time
start = time.time()

identifier = "gender"

mutant_text_1 = []
mutant_text_2 = []
prediction_1 = []
prediction_2 = []
identifier_1 = []
identifier_2 = []
names_1 = []
names_2 = []
template = []
template_id = []
for i in sgb.count().index.values :
    data = sgb.get_group(i)
    dc = data.groupby(identifier)
    for k1, v1 in dict(iter(dc)).items() :
        for k2, v2 in dict(iter(dc)).items() :
            if k1 != k2 :
                for m_1, p_1, i_1, t, t_id, n_1 in zip(v1["mutant"].values, v1["prediction"].values, v1[identifier].values, v1["template"].values, v1["template_id"].values, v1["name"].values) :
                    for m_2, p_2, i_2, n_2 in zip(v2["mutant"].values, v2["prediction"].values, v2[identifier].values, v2["name"].values) :
                        mutant_text_1.append(m_1)
                        prediction_1.append(p_1)
                        identifier_1.append(i_1)
                        mutant_text_2.append(m_2)
                        prediction_2.append(p_2)
                        identifier_2.append(i_2)
                        names_1.append(n_1)
                        names_2.append(n_2)
                        template.append(t)
                        template_id.append(t_id)
                        
end = time.time()
print("Execution time: ", end-start)

Execution time:  2.0025272369384766


In [63]:
# selected pairs
sp = pd.DataFrame(data={"mutant_1" : mutant_text_1, "mutant_2" : mutant_text_2, "prediction_1": prediction_1, "prediction_2" : prediction_2, "identifier_1": identifier_1, "name_1": names_1, "identifier_2" : identifier_2, "name_2": names_2, "template": template, "template_id": template_id})

sp

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,name_1,identifier_2,name_2,template,template_id
0,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Roen,(Warning: Some spoilers ahead.) What an incred...,51
1,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Jeet,(Warning: Some spoilers ahead.) What an incred...,51
2,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Hagen,(Warning: Some spoilers ahead.) What an incred...,51
3,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Willow,(Warning: Some spoilers ahead.) What an incred...,51
4,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Belal,(Warning: Some spoilers ahead.) What an incred...,51
...,...,...,...,...,...,...,...,...,...,...
1729795,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",1,1,male,Derick,female,Dódi,"Wow what can I say it was a good movie, very d...",5829
1729796,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",1,1,male,Derick,female,Iboyka,"Wow what can I say it was a good movie, very d...",5829
1729797,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",1,1,male,Derick,female,Zsófia,"Wow what can I say it was a good movie, very d...",5829
1729798,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",1,1,male,Derick,female,Ágota,"Wow what can I say it was a good movie, very d...",5829


In [64]:
# selected discordant pairs
sdp = sp[sp["prediction_1"] != sp["prediction_2"]]
sdp

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,name_1,identifier_2,name_2,template,template_id
0,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Roen,(Warning: Some spoilers ahead.) What an incred...,51
1,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Jeet,(Warning: Some spoilers ahead.) What an incred...,51
2,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Hagen,(Warning: Some spoilers ahead.) What an incred...,51
3,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Willow,(Warning: Some spoilers ahead.) What an incred...,51
4,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Belal,(Warning: Some spoilers ahead.) What an incred...,51
...,...,...,...,...,...,...,...,...,...,...
1729769,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",1,0,male,Derick,female,Iris,"Wow what can I say it was a good movie, very d...",5829
1729770,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",1,0,male,Derick,female,Laura,"Wow what can I say it was a good movie, very d...",5829
1729783,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",1,0,male,Derick,female,Sabine,"Wow what can I say it was a good movie, very d...",5829
1729786,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",1,0,male,Derick,female,Jocelyn,"Wow what can I say it was a good movie, very d...",5829


In [65]:
sdp["combined_name"] = sdp["name_1"] + ", " + sdp["name_2"]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [66]:
sdp = sdp.drop_duplicates()

In [67]:
sdp.groupby("combined_name").count().sort_values(by="template", ascending=False)

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,name_1,identifier_2,name_2,template,template_id
combined_name,,,,,,,,,,
"Axmed, Valentīna",25,25,25,25,25,25,25,25,25,25
"Valentīna, Axmed",25,25,25,25,25,25,25,25,25,25
"Valentīna, Igors",23,23,23,23,23,23,23,23,23,23
"Valentīna, Agnibha",23,23,23,23,23,23,23,23,23,23
"Naser , Valentīna",23,23,23,23,23,23,23,23,23,23
...,...,...,...,...,...,...,...,...,...,...
"Maria Mercedes, Waabberi",2,2,2,2,2,2,2,2,2,2
"Jeendan, Hovav",2,2,2,2,2,2,2,2,2,2
"Ertunga, Domicela",2,2,2,2,2,2,2,2,2,2


In [68]:
highest_pair = sdp.groupby("combined_name").get_group("Axmed, Valentīna")
highest_pair.head()

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,name_1,identifier_2,name_2,template,template_id,combined_name
34481,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,1,0,male,Axmed,female,Valentīna,(Warning: Some spoilers ahead.) What an incred...,51,"Axmed, Valentīna"
82531,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,0,1,male,Axmed,female,Valentīna,...and boy is the collision deafening. A femal...,90,"Axmed, Valentīna"
178631,Bad news for anyone wanting to film a full-len...,Bad news for anyone wanting to film a full-len...,0,1,male,Axmed,female,Valentīna,Bad news for anyone wanting to film a full-len...,629,"Axmed, Valentīna"
226681,"Before going any further, I have to admit that...","Before going any further, I have to admit that...",0,1,male,Axmed,female,Valentīna,"Before going any further, I have to admit that...",664,"Axmed, Valentīna"
274731,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,0,1,male,Axmed,female,Valentīna,Brass pictures (movies is not a fitting word f...,727,"Axmed, Valentīna"


In [75]:
highest_pair["male_make_sense"] = highest_pair["template_id"].apply(get_male_is_make_sense)
highest_pair["female_make_sense"] = highest_pair["template_id"].apply(get_female_is_make_sense)

highest_pair.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,name_1,identifier_2,name_2,template,template_id,combined_name,male_make_sense,female_make_sense
34481,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,1,0,male,Axmed,female,Valentīna,(Warning: Some spoilers ahead.) What an incred...,51,"Axmed, Valentīna",Yes,No
82531,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,0,1,male,Axmed,female,Valentīna,...and boy is the collision deafening. A femal...,90,"Axmed, Valentīna",No,Yes
178631,Bad news for anyone wanting to film a full-len...,Bad news for anyone wanting to film a full-len...,0,1,male,Axmed,female,Valentīna,Bad news for anyone wanting to film a full-len...,629,"Axmed, Valentīna",No,Yes
226681,"Before going any further, I have to admit that...","Before going any further, I have to admit that...",0,1,male,Axmed,female,Valentīna,"Before going any further, I have to admit that...",664,"Axmed, Valentīna",No,Yes
274731,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,0,1,male,Axmed,female,Valentīna,Brass pictures (movies is not a fitting word f...,727,"Axmed, Valentīna",No,Yes


In [69]:
import pandas as pd
import numpy as np
from bert_serving.client import BertClient
from termcolor import colored

p1 = ['Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications.', 
      'ELMo is a novel way to represent words in vectors or embeddings. These word embeddings are helpful in achieving state-of-the-art (SOTA) results in several NLP tasks: NLP scientists globally have started using ELMo for various NLP tasks, both in research as well as the industry']
p2 = ['This is a dummy example here.', 
      "But here I'm trying to say something quite different from the previous sentence."]
p3 = ['Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications.',
      'Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications.']

p1 = ["The pace of the film is ponderously slow in parts, but if you can tune into its languid speed and lengthy silences then it is a satisfying piece of courtly intrigue. The story of the first Emperor of China, his childhood sweetheart and the personal cost of power. The film is very atmospheric, the extremely mannered and polite courtly ceremony and ritual contrasted with sudden brutal violence. Filmed in a way that evokes shadows and cold spaces. Battle scenes are rare and short, the focus is on the battle within the individual on what is right to do and whether the ends justify the means. The emperor's journey from idealistic peacemaker to ruthless tyrant is aiming to be subtle, but gives little background or convincing insight into the motivation of the Emperor, indeed his actions and aims do not really change throughout, only Gong Li's attitudes to him are altered. The most interesting performances are Gong Li's and the titular Assassin as they reassess when to fight, when to retreat, when to kill. The most expensive film ever made in China at the time, the Emperor and the Assassin does not rely on hysteric emotion or big battles, but rather a brooding atmosphere of menace and inevitability. Gong Li fans will be unsurprised to hear she is as stunningly beautiful as ever, giving an understated performance.",
      "The pace of the film is ponderously slow in parts, but if you can tune into its languid speed and lengthy silences then it is a satisfying piece of courtly intrigue. The story of the first Emperor of China, his childhood sweetheart and the personal cost of power. The film is very atmospheric, the extremely mannered and polite courtly ceremony and ritual contrasted with sudden brutal violence. Filmed in a way that evokes shadows and cold spaces. Battle scenes are rare and short, the focus is on the battle within the individual on what is right to do and whether the ends justify the means. The emperor's journey from idealistic peacemaker to ruthless tyrant is aiming to be subtle, but gives little background or convincing insight into the motivation of the Emperor, indeed his actions and aims do not really change throughout, only Gong Li's attitudes to him are altered. The most interesting performances are Imre and the titular Assassin as they reassess when to fight, when to retreat, when to kill. The most expensive film ever made in China at the time, the Emperor and the Assassin does not rely on hysteric emotion or big battles, but rather a brooding atmosphere of menace and inevitability. Imre fans will be unsurprised to hear he is as stunningly beautiful as ever, giving an understated performance."]
    
Pairs = [p1]

bc = BertClient(ip='10.4.4.55', port=5555, port_out=5556, check_version=False)
    
def similarity_scoring(pair):
    import math
    query_vec_1, query_vec_2 = bc.encode(pair)
    cosine = np.dot(query_vec_1, query_vec_2) / (np.linalg.norm(query_vec_1) * np.linalg.norm(query_vec_2))
    return 1/(1 + math.exp(-100*(cosine - 0.95)))
#         return cosine

# doc_vecs = bc.encode(questions)
# print('have reached here with questions: ', questions)

print("Start testing")

for i, p in enumerate(Pairs):
    print("Similarity of Pair {}: ".format(i+1), similarity_scoring(p))

Start testing
Similarity of Pair 1:  0.9933072283262605


/opt/conda/lib/python3.6/site-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [76]:
template_id = []
template = []
valid_mutant = []
invalid_mutant = []

for index,row in highest_pair.iterrows():
    template_id.append(row["template_id"])
    template.append(row["template"])
    if row["male_make_sense"] == "Yes":
        valid_mutant.append(row["mutant_1"])
        invalid_mutant.append(row["mutant_2"])
    else :
        valid_mutant.append(row["mutant_2"])
        invalid_mutant.append(row["mutant_1"])
    

In [77]:
print(len(template))
print(len(valid_mutant))
print(len(invalid_mutant))

25
25
25


In [78]:
# valid similarity
vs = []
# not valid similarity
nvs = []

for (t,v,n) in zip(template, valid_mutant, invalid_mutant):
    vs.append(similarity_scoring([t,v]))
    nvs.append(similarity_scoring([t,n]))

/opt/conda/lib/python3.6/site-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [79]:
d = pd.DataFrame(data={"template_id": template_id, 
                       "template": template,
                       "valid_mutant" : valid_mutant,
                       "invalid_mutant" : invalid_mutant,
                       "valid_similarity" : vs,
                       "invalid_similarity" : nvs
                      })

In [80]:
d.head()

,template_id,template,valid_mutant,invalid_mutant,valid_similarity,invalid_similarity
0,51,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0.993307,0.993307
1,90,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,0.957740,0.974318
2,629,Bad news for anyone wanting to film a full-len...,Bad news for anyone wanting to film a full-len...,Bad news for anyone wanting to film a full-len...,0.993307,0.993307
3,664,"Before going any further, I have to admit that...","Before going any further, I have to admit that...","Before going any further, I have to admit that...",0.993307,0.993307
4,727,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,0.993307,0.993307


In [81]:
d["valid_similarity"].describe()

count    25.000000
mean      0.884877
std       0.287270
min       0.006415
25%       0.993307
50%       0.993307
75%       0.993307
max       0.993307
Name: valid_similarity, dtype: float64

In [82]:
d["invalid_similarity"].describe()

count    25.000000
mean      0.846676
std       0.326892
min       0.000606
25%       0.974318
50%       0.993307
75%       0.993307
max       0.993307
Name: invalid_similarity, dtype: float64

In [83]:
d["difference"] = d["valid_similarity"] - d["invalid_similarity"]

In [84]:
d.head()

,template_id,template,valid_mutant,invalid_mutant,valid_similarity,invalid_similarity,difference
0,51,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0.993307,0.993307,0.000000
1,90,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,0.957740,0.974318,-0.016578
2,629,Bad news for anyone wanting to film a full-len...,Bad news for anyone wanting to film a full-len...,Bad news for anyone wanting to film a full-len...,0.993307,0.993307,0.000000
3,664,"Before going any further, I have to admit that...","Before going any further, I have to admit that...","Before going any further, I have to admit that...",0.993307,0.993307,0.000000
4,727,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,0.993307,0.993307,0.000000


In [85]:
d["difference"].describe()

count    25.000000
mean      0.038201
std       0.120172
min      -0.016578
25%       0.000000
50%       0.000000
75%       0.000000
max       0.507936
Name: difference, dtype: float64

In [87]:
d[d["difference"] != 0]

,template_id,template,valid_mutant,invalid_mutant,valid_similarity,invalid_similarity,difference
1,90,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,0.957740,0.974318,-0.016578
5,789,Charlie Chaplin's Little Tramp or Little Man c...,Axmed Little Tramp or Little Man character wi...,Valentīna Little Tramp or Little Man characte...,0.008676,0.020969,-0.012293
9,2617,"I'm sorry, but ""Star Wars Episode 1"" did not d...","I'm sorry, but ""Star Wars Episode 1"" did not d...","I'm sorry, but ""Star Wars Episode 1"" did not d...",0.427602,0.086336,0.341266
10,2842,"In this episode, a man and his dog go 'coon hu...","In this episode, a man and his dog go 'coon hu...","In this episode, a man and her dog go 'coon hu...",0.993307,0.974874,0.018433
11,2957,It's fun to watch a young James Cagney doing h...,It's fun to watch Axmed doing his thing. He pl...,It's fun to watch Valentīna doing her thing. S...,0.006415,0.000606,0.005808
18,5020,This movie is about a man who likes to blow hi...,This movie is about a man who likes to blow hi...,This movie is about a man who likes to blow he...,0.993307,0.967332,0.025975
19,5063,"This movie is scary at times, perhaps no more ...","This movie is scary at times, perhaps no more ...","This movie is scary at times, perhaps no more ...",0.913274,0.828796,0.084478
23,5678,"When Nathaniel Kahn embarked into this voyage,...","When Axmed embarked into this voyage, he hardl...","When Valentīna embarked into this voyage, she ...",0.935392,0.427456,0.507936


In [94]:
d[d["template_id"] == 5678]["invalid_mutant"].values

array(["When Valentīna embarked into this voyage, she hardly knew who her father really was. By the end of the film, she found her and comes to terms with the strange life she lived as a child. Valentīna was the father. She was an architect's architect. Her designs were perhaps too complex, as she tried to create buildings that didn't conform with trends popular at that time. It is ironic that she never achieved the fame that came so easy to some of her contemporaries. She had a vision and she never strayed from it. We can see characteristics of her unique style in the buildings she left behind as a legacy to humanity. Every one of her creations are unique in that they don't imitate works from other architects. Valentīna Valentīna life was rather complicated. She was married, yet she had affairs with two of her assistants that produced a girl and a boy, besides the legitimate daughter she had with her wife. As a boy, Valentīna life was lived in a secluded area, away from her father, wh